## Miami-Dade County Midterm Election Results — Governor and Lieutenant Governor (2022)

By: Shirsho Dasgupta (2022)

The code in this repository scrapes precinct-level election results for the Governor race in Miami-Dade county.

The column names in the dataframe are generated only after the results have been scraped, meaning that the code is reusable for different races and the dataframe dynamically sizes itself according to the number of candidates or issues on the ballot. It does not matter how many candidates there are or in which order they appear.

The code then computes the vote shares of each candidate, generates the names of the candidates who are leading (or have won) and are runners-up, their party affiliations and their lead/win margin in each precinct. It also computes the leader/winner and their vote-share and lead/win margin for the entire county.

The resulting dataframe for each race is then joined with previously prepared spreadsheets containing precinct-level demographic and voter registration data.



### Importing libraries

In [1]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd
import datetime as dt
import os
import re

In [2]:
### ignores copy warning
pd.options.mode.chained_assignment = None 

### Downloading precinct-level report and setting dataframe

In [3]:
### creates directory to download report from website
os.makedirs("governor_dade/reports_raw/", exist_ok = True)
### creates directory to store final dataframe after exporting
os.makedirs("governor_dade/report_files/", exist_ok = True)

### sets URL
url = "https://enr.electionsfl.org/DAD/3267/Reports/"

### retrieves data
page = requests.get(url)
### converts data to text for reading
soup = BeautifulSoup(page.text)

### stores all a-tags with links in them
a_report = soup.find_all("a", href = True)

### retrieves the download link
file_link = a_report[-2].get("href")
### stores time of access
tnow = dt.datetime.now()
### constructs file name
filename = "raw_report_" + str(tnow.strftime("%m-%d-%Y-%H%M")) + ".csv"

### parses data
r = requests.get(file_link)

### checks if file is available to download, if not it prints an error message
if r.status_code != 200:
    print(f"Error downloading {file_link}")
### downloads file
else:
    with open("governor_dade/reports_raw/" + filename, "wb") as outfile:
        outfile.write(r.content)

### imports the file as dataframe
df_raw = pd.read_csv("governor_dade/reports_raw/" + filename)

### displays dataframe
df_raw

,Contest,Candidate Issue,Party,Precinct Name,Mail Votes,Early Votes,Election Day Votes,Total Votes,% of Total Votes
0,United States Senator,Marco Rubio,REP,PRECINCT 001.0,118,111,70,299,69.37
1,United States Senator,Marco Rubio,REP,PRECINCT 010.0,-,-,-,32,91.43
2,United States Senator,Marco Rubio,REP,PRECINCT 100.0,189,0,0,189,26.00
3,United States Senator,Marco Rubio,REP,PRECINCT 100.2,43,0,0,43,14.93
4,United States Senator,Marco Rubio,REP,PRECINCT 100.3,64,0,0,64,15.35
...,...,...,...,...,...,...,...,...,...
50671,SMiami Installation Elected Official,No,NaN,PRECINCT 651.0,-,-,-,73,23.47
50672,SMiami Installation Elected Official,No,NaN,PRECINCT 652.0,-,-,-,70,20.90
50673,SMiami Installation Elected Official,No,NaN,PRECINCT 653.0,-,-,-,63,14.79
50674,SMiami Installation Elected Official,No,NaN,PRECINCT 653.1,-,-,-,4,15.38


### Filtering on relevant election-type

In [4]:
for i in range(0, len(df_raw)):
    df_raw["Contest"][i] = str(df_raw["Contest"][i].upper())

### filters for specific contest
df_filter = df_raw[df_raw["Contest"] == "GOVERNOR AND LIEUTENANT GOVERNOR"].reset_index(drop = True)

### loop runs through filtered dataframe
for i in range(0, len(df_filter)):
    ### converts party to string type (null values were being read as float)
    df_filter["Party"][i] = str(df_filter["Party"][i])
    ### replaces with "OTHER"
    df_filter["Party"][i] = df_filter["Party"][i].replace("nan", "OTHER")
    df_filter["Candidate Issue"][i] = df_filter["Candidate Issue"][i].replace("WRITE-IN", "Write-In")

### displays dataframe
df_filter

,Contest,Candidate Issue,Party,Precinct Name,Mail Votes,Early Votes,Election Day Votes,Total Votes,% of Total Votes
0,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 001.0,118,111,74,303,70.47
1,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 010.0,-,-,-,29,82.86
2,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 100.0,185,0,0,185,25.77
3,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 101.0,211,260,205,676,48.67
4,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 102.0,451,579,278,1308,54.75
...,...,...,...,...,...,...,...,...,...
4191,GOVERNOR AND LIEUTENANT GOVERNOR,Carmen Jackie Gimenez,NPA,PRECINCT 999.0,-,-,-,1,2.00
4192,GOVERNOR AND LIEUTENANT GOVERNOR,Carmen Jackie Gimenez,NPA,PRECINCT 999.1,-,-,-,1,0.82
4193,GOVERNOR AND LIEUTENANT GOVERNOR,Carmen Jackie Gimenez,NPA,PRECINCT 999.2,0,0,0,0,0.00
4194,GOVERNOR AND LIEUTENANT GOVERNOR,Carmen Jackie Gimenez,NPA,PRECINCT 999.3,0,0,0,0,0.00


### Extracting base precincts and compiling results into precinct-level

Note: Comment out this block if compilation is not needed. Add a command saying the following:
df_voteprecinct = df_filter.copy()

In [5]:
### creates new columns to be written into
df_filter["Precinct"] = " "
df_filter["Base_Precinct"] = " "

### loop runs through dataframe
for i in range(0, len(df_filter)):
    ### extracts precinct number and stores as a list
    precinct = re.findall('\d*\.?\d+', df_filter["Precinct Name"][i])
    ### extracts precinct number from list (in this case list has only one element) and writes into cell
    df_filter["Precinct"][i] = precinct[0]
    ### extracts base precinct number and writes into cell
    df_filter["Base_Precinct"][i] = int(df_filter["Precinct"][i][:3])

### displays dataframe
df_filter

,Contest,Candidate Issue,Party,Precinct Name,Mail Votes,Early Votes,Election Day Votes,Total Votes,% of Total Votes,Precinct,Base_Precinct
0,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 001.0,118,111,74,303,70.47,001.0,1
1,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 010.0,-,-,-,29,82.86,010.0,10
2,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 100.0,185,0,0,185,25.77,100.0,100
3,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 101.0,211,260,205,676,48.67,101.0,101
4,GOVERNOR AND LIEUTENANT GOVERNOR,Ron DeSantis,REP,PRECINCT 102.0,451,579,278,1308,54.75,102.0,102
...,...,...,...,...,...,...,...,...,...,...,...
4191,GOVERNOR AND LIEUTENANT GOVERNOR,Carmen Jackie Gimenez,NPA,PRECINCT 999.0,-,-,-,1,2.00,999.0,999
4192,GOVERNOR AND LIEUTENANT GOVERNOR,Carmen Jackie Gimenez,NPA,PRECINCT 999.1,-,-,-,1,0.82,999.1,999
4193,GOVERNOR AND LIEUTENANT GOVERNOR,Carmen Jackie Gimenez,NPA,PRECINCT 999.2,0,0,0,0,0.00,999.2,999
4194,GOVERNOR AND LIEUTENANT GOVERNOR,Carmen Jackie Gimenez,NPA,PRECINCT 999.3,0,0,0,0,0.00,999.3,999


In [6]:
### converts number of votes into integers
df_filter["Total Votes"] = df_filter["Total Votes"].astype(int)
### groups sub-precincts into base precincts
df_voteprecinct = df_filter.groupby(["Base_Precinct", "Candidate Issue", "Party"]).agg({"Total Votes": "sum"}).reset_index()

### loop runs through newly created dataframe
for i in range(0, len(df_voteprecinct)):
    ### converts base precinct numbers into strings
    df_voteprecinct["Base_Precinct"][i] = str(df_voteprecinct["Base_Precinct"][i])

### displays dataframe
df_voteprecinct

,Base_Precinct,Candidate Issue,Party,Total Votes
0,1,Carmen Jackie Gimenez,NPA,0
1,1,Charlie Crist,DEM,127
2,1,Hector Roos,LPF,0
3,1,Ron DeSantis,REP,303
4,2,Carmen Jackie Gimenez,NPA,1
...,...,...,...,...
3127,997,Ron DeSantis,REP,136
3128,999,Carmen Jackie Gimenez,NPA,2
3129,999,Charlie Crist,DEM,158
3130,999,Hector Roos,LPF,0


### Converting datasets into horizontal format

In [7]:
### stores each precinct number
precincts = df_voteprecinct["Base_Precinct"].explode().unique() 

In [8]:
### stores names of each candidate
candidates = df_voteprecinct["Candidate Issue"].explode().unique() 
### computes and stores number of candidates
candidate_len = len(candidates)

In [9]:
### initializes new list (referred to as "final list" later) to write data into
final_rows = []

### loop runs through dataframe
for i in range(0, len(precincts)):
    
    ### stores time of processing/update
    tnow = dt.datetime.now()
    dt_string = tnow.strftime("%m-%d-%Y %H:%M")
    update = "Last updated at " + str(dt_string)
    
    ### creates new list with first element as processing time
    pre_row = [update]
    
    ### stores base precinct number
    precinct = precincts[i]
    
    ### adds base precinct number to list
    pre_row.append(precinct)
    
    ### nested loop runs through dataframe 
    for j in range(0, len(df_voteprecinct)):
        
        ### if the base precinct number matches, then candidate details are stored in list
        if df_voteprecinct["Base_Precinct"][j] == precinct:
            
            ### adds party to the candidate name in format <NAME (PARTY)>
            can_name = df_voteprecinct["Candidate Issue"][j] + " (" + str(df_voteprecinct["Party"][j]) + ")" 
            
            ### adds candidate name to list
            pre_row.append(can_name)
            ### adds votes won to list
            pre_row.append(df_voteprecinct["Total Votes"][j])
            ### adds party name to list
            pre_row.append(df_voteprecinct["Party"][j])
            
            ### adds list to final list
            final_rows.append(pre_row)
        else:
            continue
    
    ### converts final list to dataframe
    df_initial = pd.DataFrame(final_rows)

### displays dataframe
df_initial

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),303,REP
1,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),303,REP
2,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),303,REP
3,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),303,REP
4,Last updated at 02-13-2023 22:46,2,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),254,DEM,Hector Roos (LPF),2,LPF,Ron DeSantis (REP),597,REP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,Last updated at 02-13-2023 22:46,997,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),54,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),136,REP
3128,Last updated at 02-13-2023 22:46,999,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),158,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),107,REP
3129,Last updated at 02-13-2023 22:46,999,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),158,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),107,REP
3130,Last updated at 02-13-2023 22:46,999,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),158,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),107,REP


In [10]:
### removes duplicates
df = df_initial[::candidate_len].reset_index(drop = True)

### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),303,REP
1,Last updated at 02-13-2023 22:46,2,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),254,DEM,Hector Roos (LPF),2,LPF,Ron DeSantis (REP),597,REP
2,Last updated at 02-13-2023 22:46,3,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),353,DEM,Hector Roos (LPF),2,LPF,Ron DeSantis (REP),1042,REP
3,Last updated at 02-13-2023 22:46,4,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),331,DEM,Hector Roos (LPF),1,LPF,Ron DeSantis (REP),717,REP
4,Last updated at 02-13-2023 22:46,5,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),560,DEM,Hector Roos (LPF),4,LPF,Ron DeSantis (REP),1090,REP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-13-2023 22:46,994,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),151,DEM,Hector Roos (LPF),3,LPF,Ron DeSantis (REP),290,REP
779,Last updated at 02-13-2023 22:46,995,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),1022,DEM,Hector Roos (LPF),3,LPF,Ron DeSantis (REP),1031,REP
780,Last updated at 02-13-2023 22:46,996,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),683,DEM,Hector Roos (LPF),4,LPF,Ron DeSantis (REP),469,REP
781,Last updated at 02-13-2023 22:46,997,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),54,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),136,REP


### Computing total votes tallied

In [11]:
### storing number of columns
col_no = len(df.columns)

### creates new column to write totals into at the end
### since this is an index-number, it automatically computes the required column number irrespective of number of candidates
df[col_no] = " "

### creates list to store tallied votes
votes_tallied = []

### loop runs through newly created horizontal dataframe
for i in range(0, len(df)):
    
    ### initializes variable to store total
    total = 0
    
    ### nested loop starts at column 3 (that is the first column with a vote number)
    ### loop iterates (number of candidates*3 + 1) at intervals of three to read all tallied votes
    ### note: each candidate has three corresponding elements—name,votes,party—hence the (n*3+1)
    for j in range(3, (candidate_len*3 + 1), 3):
        ### adds votes to total
        total = total + df[j][i]
    
    ### adds total to list
    votes_tallied.append(total)
    ### writes total into new column
    df[col_no][i] = total

### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),303,REP,430
1,Last updated at 02-13-2023 22:46,2,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),254,DEM,Hector Roos (LPF),2,LPF,Ron DeSantis (REP),597,REP,854
2,Last updated at 02-13-2023 22:46,3,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),353,DEM,Hector Roos (LPF),2,LPF,Ron DeSantis (REP),1042,REP,1401
3,Last updated at 02-13-2023 22:46,4,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),331,DEM,Hector Roos (LPF),1,LPF,Ron DeSantis (REP),717,REP,1057
4,Last updated at 02-13-2023 22:46,5,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),560,DEM,Hector Roos (LPF),4,LPF,Ron DeSantis (REP),1090,REP,1657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-13-2023 22:46,994,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),151,DEM,Hector Roos (LPF),3,LPF,Ron DeSantis (REP),290,REP,447
779,Last updated at 02-13-2023 22:46,995,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),1022,DEM,Hector Roos (LPF),3,LPF,Ron DeSantis (REP),1031,REP,2060
780,Last updated at 02-13-2023 22:46,996,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),683,DEM,Hector Roos (LPF),4,LPF,Ron DeSantis (REP),469,REP,1157
781,Last updated at 02-13-2023 22:46,997,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),54,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),136,REP,190


### Computing precinct-level vote shares, leads and margins

In [12]:
### creates new list to store index numbers of newly created columns
new_cols = []

### loop runs through number of candidates and four more for other details
for i in range(0, (candidate_len + 4)):   
    ### creates (n+4) new columns for "n" candidates
    df[col_no + i + 1] = " "
    ### stores numbers in a list as strings
    new_cols.append(str(col_no + i + 1))

### loop runs through each row of dataframe
for i in range(0, len(df)):
    
    ### creates lists to store details
    votes_obtained = []
    party = []
    can_names = []
    
    ### stores the total votes tallied for each row
    total = votes_tallied[i]
    
    if total != 0: 
    
        ## TALLYING VOTES

        ### nested loop starts at column 3 (that is the first column with a vote number)
        ### loop iterates (number of candidates*3 + 1) at intervals of three to read all details
        ### note: each candidate has three corresponding elements—name,votes,party—hence the (n*3+1)
        for j in range(3, (candidate_len*3 + 1), 3):

            ### stores votes, party and candidate names in the relevant lists
            ### note: here format is [cols x rows] while traversing through dataframe
            votes_obtained.append(df[j][i])
            party.append(df[j+1][i])
            can_names.append(df[j-1][i])


        ## COMPUTING VOTE-SHARES

        ### loop runs through all the candidates
        for k in range(0, candidate_len):
            ### column-index of a candidate is given by corresponding element in "new_cols"
            ### first candidate (k = 0) will be stored in column at index-0 in the "new_cols" list and so on
            column_no = int(new_cols[k])
            ### computes percent and writes into the column
            df[column_no][i] = (votes_obtained[k]/total*100).round(2)

        ## COMPUTING LEADING CANDIDATES AND WIN MARGINS

        ### computes and stores the highest vote-tally for the "votes_obtained" list
        max_value = max(votes_obtained)

        ### if no votes have been tallied
        #if max_value == 0:

            #lead_party = "N/A"
            #lead_can = "No votes tallied."
            #margin = 0
            #margin_pp = 0

        ### checks if there are more elements with same value (in case of ties)
        if votes_obtained.count(max_value) > 1:

            ### method returns indices of all duplicate highest vote-tallies
            indices = []
            def find_indices(list_to_check, item_to_find):
                return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

            ### stores indices of all duplicate highest vote-tallies
            indices = find_indices(votes_obtained, max_value)

            ### constructs string of all candidate names corresponding to highest vote-tallies
            string = []
            for ind in range(0, len(indices)):
                index = indices[ind]
                string.append(can_names[index])

            ### stores value as "TIED"
            lead_party = "TIED"
            ### stores candidate names with tied votes
            lead_can = " and ".join([", ".join(string[:-1]),string[-1]] if len(string) > 2 else string)
            ### stores margin of difference as 0
            margin = 0
            margin_pp = 0

        else:
            ### computes and stores the index at which the highest vote is stored in the "votes_obtained" list
            max_index = votes_obtained.index(max_value)
            ### stores name of party corresponding to highest vote-tally in the fourth-last column
            ### if highest vote-tally is second (index=1) element in "votes_obtained"
            ### the second element (index=1) in "party" list is the corresponding datapoint
            lead_party = party[max_index]
            ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
            lead_can = can_names[max_index]

            ### computes and stores the second highest vote-tally for the "votes_obtained" list
            second_max_value = max(votes_obtained, key = lambda x: min(votes_obtained)-1 if (x == max_value) else x)
            ### computes and stores the raw number of votes between first and second placed candidates
            margin = max_value - second_max_value
            ### computes the difference in percent points between first and second placed candidates
            margin_pp = ((max_value/total*100)-(second_max_value/total*100)).round(2)

        ### stores index-number of the fourth-last element of "new_cols" list (this will be the fourth-last column)
        lead_col = int(new_cols[-4])
        ### stores name of party corresponding to highest vote-tally in the fourth-last column
        ### if highest vote-tally is second (index=1) element in "votes_obtained"
        ### the second element (index=1) in "party" list is the corresponding datapoint
        df[lead_col][i] = lead_party
        ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
        df[lead_col+1][i] = lead_can

        ### stores index-number of the penultimate element of "new_cols" list (this will be the penultimate column)
        raw_margin_col = int(new_cols[-2])
        ### stores raw margin of difference in penultimate column
        df[raw_margin_col][i] = margin

        ### stores index-number of the last element of "new_cols" list (this will be the last column)
        margin_pp_col = int(new_cols[-1])
        ### stores percent points difference in last column
        df[margin_pp_col][i] = margin_pp
    
    else: 
        
        ### loop runs through all candidates
        for n in range(0, candidate_len):
            ### column-index of a candidate is given by corresponding element in "new_cols"
            ### first candidate (k = 0) will be stored in column at index-0 in the "new_cols" list and so on
            column_no = int(new_cols[n])
            ### computes percent and writes into the column
            df[column_no][i] = 0
        
        lead_party = "No votes tallied"
        lead_can = "No votes tallied"
        margin = 0
        margin_pp = 0
        
        ### stores index-number of the fourth-last element of "new_cols" list (this will be the fourth-last column)
        lead_col = int(new_cols[-4])
        ### stores name of party corresponding to highest vote-tally in the fourth-last column
        ### if highest vote-tally is second (index=1) element in "votes_obtained"
        ### the second element (index=1) in "party" list is the corresponding datapoint
        df[lead_col][i] = lead_party
        ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
        df[lead_col+1][i] = lead_can

        ### stores index-number of the penultimate element of "new_cols" list (this will be the penultimate column)
        raw_margin_col = int(new_cols[-2])
        ### stores raw margin of difference in penultimate column
        df[raw_margin_col][i] = margin

        ### stores index-number of the last element of "new_cols" list (this will be the last column)
        margin_pp_col = int(new_cols[-1])
        ### stores percent points difference in last column
        df[margin_pp_col][i] = margin_pp       

### replaces null values with "0"
df.fillna(0, inplace = True)    
   
### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,...,REP,430,0.00,29.53,0.00,70.47,REP,Ron DeSantis (REP),176,40.93
1,Last updated at 02-13-2023 22:46,2,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),254,DEM,Hector Roos (LPF),2,...,REP,854,0.12,29.74,0.23,69.91,REP,Ron DeSantis (REP),343,40.16
2,Last updated at 02-13-2023 22:46,3,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),353,DEM,Hector Roos (LPF),2,...,REP,1401,0.29,25.20,0.14,74.38,REP,Ron DeSantis (REP),689,49.18
3,Last updated at 02-13-2023 22:46,4,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),331,DEM,Hector Roos (LPF),1,...,REP,1057,0.76,31.32,0.09,67.83,REP,Ron DeSantis (REP),386,36.52
4,Last updated at 02-13-2023 22:46,5,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),560,DEM,Hector Roos (LPF),4,...,REP,1657,0.18,33.80,0.24,65.78,REP,Ron DeSantis (REP),530,31.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-13-2023 22:46,994,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),151,DEM,Hector Roos (LPF),3,...,REP,447,0.67,33.78,0.67,64.88,REP,Ron DeSantis (REP),139,31.10
779,Last updated at 02-13-2023 22:46,995,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),1022,DEM,Hector Roos (LPF),3,...,REP,2060,0.19,49.61,0.15,50.05,REP,Ron DeSantis (REP),9,0.44
780,Last updated at 02-13-2023 22:46,996,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),683,DEM,Hector Roos (LPF),4,...,REP,1157,0.09,59.03,0.35,40.54,DEM,Charlie Crist (DEM),214,18.50
781,Last updated at 02-13-2023 22:46,997,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),54,DEM,Hector Roos (LPF),0,...,REP,190,0.00,28.42,0.00,71.58,REP,Ron DeSantis (REP),82,43.16


### Computing overall county vote shares, leads and margins

In [13]:
## COMPUTING COUNTY-LEVEL TOTALS

### storing number of columns
new_col_no = len(df.columns)

total_col = candidate_len*3 + 2
df[new_col_no] = df[total_col].sum() 
ct_total = df[total_col].sum()

### creates new list to store index numbers of newly created columns
ct_cols = []

### stores new length of dataframe
ct_new_col_no = len(df.columns)

if ct_total != 0:

    ## COMPUTING COUNTY-LEVEL VOTE SHARES

    ### computes sum of all votes per candidate
    ct_can_votes = []
    for j in range(3, (candidate_len*3 + 1), 3):
        ct_can_votes.append(df[j].sum())

    ### computes vote-shares of candidates
    for i in range(0, candidate_len):
        df[ct_new_col_no + i] = (ct_can_votes[i]/ct_total*100).round(2)

    ## COMPUTING LEADING CANDIDATES AND WIN MARGINS

    ### computes new length of dataframe
    final_col_no = len(df.columns)

    ### stores overall maximum votes
    ct_max_value = max(ct_can_votes)

    ### if no votes have been tallied
    #if ct_max_value == 0:

        #ct_lead_party = "N/A"
        #ct_lead_can = "No votes tallied."
        #ct_margin = 0
        #ct_margin_pp = 0

    ### if there are more than one instances of highest votes (in case of a tie)
    if ct_can_votes.count(ct_max_value) > 1:

        ### method returns indices of all duplicate highest vote-tallies
        indices = []
        def find_indices(list_to_check, item_to_find):
            return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

        ### stores indices of all duplicate highest vote-tallies
        indices = find_indices(ct_can_votes, ct_max_value)

        ### constructs string of all candidate names corresponding to highest votes tallied
        string = []
        for ind in range(0, len(indices)):
            index = indices[ind]
            string.append(can_names[index])

        ### stores leading party as tied
        ct_lead_party = "TIED"
        ### stores names of tied candidates
        ct_lead_can = " and ".join([", ".join(string[:-1]),string[-1]] if len(string) > 2 else string)

        ### stores margins as "0"
        ct_margin = 0
        ct_margin_pp = 0

        ### stores vote share for candidate with highest vote-tally
        ct_lead_voteshare = (ct_max_value/ct_total*100).round(2)

    else: 

        ### computes index of the highest tallied vote
        ct_max_index = ct_can_votes.index(ct_max_value)

        ### stores name of corresponding party and candidates
        ct_lead_party = party[ct_max_index]
        ct_lead_can = can_names[ct_max_index]

        ### computes second highest tallied vote
        second_ct_max_value = max(ct_can_votes, key = lambda x: min(ct_can_votes)-1 if (x == ct_max_value) else x)
        ### computes index of the second-highest tallied vote
        second_ct_max_index = ct_can_votes.index(second_ct_max_value)
        
        ### stores name of corresponding party and candidates
        second_ct_party = party[second_ct_max_index]
        second_ct_can = can_names[second_ct_max_index]

        ### stores margin of difference and the margin as percent points
        ct_margin = ct_max_value - second_ct_max_value
        ct_margin_pp = ((ct_max_value/ct_total*100)-(second_ct_max_value/ct_total*100)).round(2)

        ### stores the vote share of the highest tallied vote
        ct_lead_voteshare = (ct_max_value/ct_total*100).round(2)
        ### stores the vote share of the second highest tallied vote
        second_ct_voteshare = (second_ct_max_value/ct_total*100).round(2)

    ### writes into cells
    df[final_col_no + 1] = ct_lead_party
    df[final_col_no + 2] = ct_lead_can
    df[final_col_no + 3] = ct_margin
    df[final_col_no + 4] = ct_margin_pp
    df[final_col_no + 5] = ct_lead_voteshare
    df[final_col_no + 6] = second_ct_party
    df[final_col_no + 7] = second_ct_can
    df[final_col_no + 8] = second_ct_voteshare
    
else: 
    
    ### sets vote-shares of candidates as 0
    for i in range(0, candidate_len):
        df[ct_new_col_no + i] = 0
        
    ### computes new length of dataframe
    final_col_no = len(df.columns)
    
    ct_lead_party = "No votes tallied"
    ct_lead_can = "No votes tallied"
    ct_margin = 0
    ct_margin_pp = 0
    ct_lead_voteshare = 0
    second_ct_party = "No votes tallied"
    second_ct_can = "No votes tallied"
    second_ct_voteshare = 0
    
    ### writes into cells
    df[final_col_no + 1] = ct_lead_party
    df[final_col_no + 2] = ct_lead_can
    df[final_col_no + 3] = ct_margin
    df[final_col_no + 4] = ct_margin_pp
    df[final_col_no + 5] = ct_lead_voteshare
    df[final_col_no + 6] = second_ct_party
    df[final_col_no + 7] = second_ct_can
    df[final_col_no + 8] = second_ct_voteshare

### replaces null values with "0"
df.fillna(0, inplace = True)    
   
### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,26,27,29,30,31,32,33,34,35,36
0,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
1,Last updated at 02-13-2023 22:46,2,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),254,DEM,Hector Roos (LPF),2,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
2,Last updated at 02-13-2023 22:46,3,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),353,DEM,Hector Roos (LPF),2,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
3,Last updated at 02-13-2023 22:46,4,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),331,DEM,Hector Roos (LPF),1,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
4,Last updated at 02-13-2023 22:46,5,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),560,DEM,Hector Roos (LPF),4,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-13-2023 22:46,994,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),151,DEM,Hector Roos (LPF),3,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
779,Last updated at 02-13-2023 22:46,995,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),1022,DEM,Hector Roos (LPF),3,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
780,Last updated at 02-13-2023 22:46,996,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),683,DEM,Hector Roos (LPF),4,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
781,Last updated at 02-13-2023 22:46,997,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),54,DEM,Hector Roos (LPF),0,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97


### Computing and inserting column names

In [14]:
### creates list to store column names with two initial elements
cols = ["LAST_UPDATE_2022", "BASE_PRECINCT"]

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column names for each candidate and adds to list
    cols.append("CANDIDATE_NAME_" + str(i+1))
    cols.append("VOTES_WON_" + str(i+1))
    cols.append("CANDIDATE_PARTY_" + str(i+1))

### adds column to list
cols.append("TOTAL_VOTES_TALLIED")

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column name for each candidate and adds to list
    cols.append("VOTE_SHARE_" + str(i+1))

### adds columns to list
cols.append("LEAD_PARTY")
cols.append("LEAD_CANDIDATE")
cols.append("RAW_MARGIN")
cols.append("MARGIN_PERCENTPOINTS")
cols.append("CT_TOTAL_VOTES_TALLIED")

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column names for each candidate and adds to list
    cols.append("CT_VOTE_SHARE_" + str(i+1))

cols.append("CT_LEAD_PARTY")
cols.append("CT_LEAD_CANDIDATE")
cols.append("CT_RAW_MARGIN")
cols.append("CT_MARGIN_PERCENTPOINTS")
cols.append("CT_LEAD_VOTESHARE")
cols.append("CT_SECOND_PARTY")
cols.append("CT_SECOND_CANDIDATE")
cols.append("CT_SECOND_VOTESHARE")

### replaces column indices with generated column names
df.columns = cols

### displays dataframe
df

,LAST_UPDATE_2022,BASE_PRECINCT,CANDIDATE_NAME_1,VOTES_WON_1,CANDIDATE_PARTY_1,CANDIDATE_NAME_2,VOTES_WON_2,CANDIDATE_PARTY_2,CANDIDATE_NAME_3,VOTES_WON_3,...,CT_VOTE_SHARE_3,CT_VOTE_SHARE_4,CT_LEAD_PARTY,CT_LEAD_CANDIDATE,CT_RAW_MARGIN,CT_MARGIN_PERCENTPOINTS,CT_LEAD_VOTESHARE,CT_SECOND_PARTY,CT_SECOND_CANDIDATE,CT_SECOND_VOTESHARE
0,Last updated at 02-13-2023 22:46,1,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
1,Last updated at 02-13-2023 22:46,2,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),254,DEM,Hector Roos (LPF),2,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
2,Last updated at 02-13-2023 22:46,3,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),353,DEM,Hector Roos (LPF),2,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
3,Last updated at 02-13-2023 22:46,4,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),331,DEM,Hector Roos (LPF),1,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
4,Last updated at 02-13-2023 22:46,5,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),560,DEM,Hector Roos (LPF),4,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-13-2023 22:46,994,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),151,DEM,Hector Roos (LPF),3,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
779,Last updated at 02-13-2023 22:46,995,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),1022,DEM,Hector Roos (LPF),3,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
780,Last updated at 02-13-2023 22:46,996,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),683,DEM,Hector Roos (LPF),4,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97
781,Last updated at 02-13-2023 22:46,997,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),54,DEM,Hector Roos (LPF),0,...,0.24,55.28,REP,Ron DeSantis (REP),80560,11.32,55.28,DEM,Charlie Crist (DEM),43.97


### Merging final dataframe with dataset of precinct demographics and prior results

In [15]:
### importing dataset with demographics and prior results
df_prior = pd.read_csv("dade_gov_prior.csv", keep_default_na = False)

### merging 
df_master = pd.merge(df, df_prior, on = "BASE_PRECINCT")

df_master["BASE_PRECINCT"] = df_master["BASE_PRECINCT"].apply(lambda x: "{0:0>3}".format(x))
#df_master["BASE_PRECINCT"] = str(df_master["BASE_PRECINCT"])

### loop runs through dataframe
for i in range(0, len(df_master)):
    ### converts the precinct numbers to string
    df_master["BASE_PRECINCT"][i] = str(df_master["BASE_PRECINCT"][i])
    
### stores time
dt_string = tnow.strftime("%m-%d-%Y-%H%M")
### constructs filename
report = "gov_dade_report_" + str(dt_string) + ".csv"
path = "governor_dade/report_files/" + report

### exports dataframe
df_master.to_csv(path, index = False)

### displays dataframe
df_master

,LAST_UPDATE_2022,BASE_PRECINCT,CANDIDATE_NAME_1,VOTES_WON_1,CANDIDATE_PARTY_1,CANDIDATE_NAME_2,VOTES_WON_2,CANDIDATE_PARTY_2,CANDIDATE_NAME_3,VOTES_WON_3,...,TOTAL_POP,RACE,PARTISANSHIP,DEMOCRAT_SHARE,REPUBLICAN_SHARE,WHITE_SHARE,BLACK_SHARE,HISPANIC_SHARE,OTHER_SHARE,MUNICIPALITY
0,Last updated at 02-13-2023 22:46,001,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),127,DEM,Hector Roos (LPF),0,...,762,Majority White,Other,26.4,32.5,65,0.3,25.9,8.9,Golden Beach
1,Last updated at 02-13-2023 22:46,002,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),254,DEM,Hector Roos (LPF),2,...,1810,Majority White,Other,25.6,31.5,51.5,1.3,35.7,11.4,Sunny Isles Beach
2,Last updated at 02-13-2023 22:46,003,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),353,DEM,Hector Roos (LPF),2,...,2689,Majority White,Other,25.5,34.5,67.1,1.2,22.6,9.1,Sunny Isles Beach
3,Last updated at 02-13-2023 22:46,004,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),331,DEM,Hector Roos (LPF),1,...,2753,Other,Other,26.6,29,44.3,5,39.4,11.3,Sunny Isles Beach
4,Last updated at 02-13-2023 22:46,005,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),560,DEM,Hector Roos (LPF),4,...,3427,Majority White,Other,24.2,31.5,54.8,2.6,31.7,10.9,Sunny Isles Beach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,Last updated at 02-13-2023 22:46,994,Carmen Jackie Gimenez (NPA),3,NPA,Charlie Crist (DEM),151,DEM,Hector Roos (LPF),3,...,856,Majority Hispanic,Other,26.1,39.3,12.6,0.7,79.9,6.8,Miami
779,Last updated at 02-13-2023 22:46,995,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),1022,DEM,Hector Roos (LPF),3,...,4980,Other,Other,30.5,25.3,46.7,3.8,37.3,12.1,Miami
780,Last updated at 02-13-2023 22:46,996,Carmen Jackie Gimenez (NPA),1,NPA,Charlie Crist (DEM),683,DEM,Hector Roos (LPF),4,...,3082,Other,Other,34.3,21.7,48.4,4.8,32.7,14,Miami
781,Last updated at 02-13-2023 22:46,997,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),54,DEM,Hector Roos (LPF),0,...,481,Majority Hispanic,Other,30.6,32.4,2.7,0.8,89.2,7.3,Miami
